In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2
from sklearn.linear_model import Lasso
# import xgboost as xgb
#DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from typing import List, Dict,Tuple,Any
from scipy.stats import chi2_contingency
import scipy.stats as stats

# Pipeline imports
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#import LogisticRegression
from sklearn.linear_model import LogisticRegression
#import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import TransformerMixin
#impory accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

import pickle

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2
from sklearn.model_selection import train_test_split

#ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [2]:
#list names of all files
os.listdir('data')


['leads.csv', 'rates_land.csv', 'rates_sea.csv', 'rates_air.csv']

In [3]:
#read the data
list_of_files = os.listdir('data')
leads, air, land, sea = [pd.read_csv('data/' + file) for file in list_of_files]

In [4]:
leads


,request_id,request_date,valid_date_from,valid_date_to,weight,weight_unit,amount,container_type,shipping_type,hs_code,...,liquid_cargo,imo_code,city_from_country,city_from,port_from_country,port_from,port_to_country,port_to,city_to_country,city_to
0,855641,2023-09-05,2023-09-16,2023-09-23,26.63,Metric Ton,1.0,40' Refrigerated,FCL,06-14,...,False,NaN,NaN,NaN,ID,Jakarta,BD,Chittagong,NaN,NaN
1,855652,2023-09-05,2023-09-30,2023-10-07,28.00,Metric Ton,1.0,20' Standard,FCL,25-27,...,False,NaN,NaN,NaN,BR,Vitoria,US,Houston,US,"Houston, TX, USA"
2,855664,2023-09-05,2023-09-05,2023-09-12,200.00,Metric Ton,1.0,20' Standard,FCL,NaN,...,False,NaN,NaN,NaN,CN,Chengdu,DO,Caucedo,DO,Dominican Republic
3,855689,2023-09-05,2023-09-30,2023-10-07,26.00,Metric Ton,20.0,40' Standard,FCL,0910,...,False,NaN,NaN,NaN,NG,Apapa,AE,Jebel Ali,AE,Mina Jebel Ali - Dubai - United Arab Emirates
4,855725,2023-09-05,2023-09-20,2023-09-27,250.00,Metric Ton,NaN,NaN,LCL,8711,...,False,NaN,NaN,NaN,IN,Chennai,DE,FRANKFURT AM MAIN,DE,"Frankfurt, Germany"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9893,941678,2024-03-05,2024-06-09,2024-06-16,30.00,Kilogram,NaN,NaN,LCL,NaN,...,False,NaN,IT,"Padua, Province of Padua, Italy",IT,Venice,CA,Toronto,CA,"Toronto, ON, Canada"
9894,941682,2024-03-05,2024-03-05,2024-03-12,28.23,Metric Ton,1.0,20' Standard,FCL,94-96,...,False,NaN,NaN,NaN,BR,Rio De Jan Eiro,SG,Singapore,NaN,NaN
9895,941683,2024-03-05,2024-06-09,2024-06-09,310.00,Kilogram,NaN,NaN,LCL,NaN,...,False,NaN,IT,"Padua, Province of Padua, Italy",IT,Venice,CA,Toronto,CA,"Toronto, ON, Canada"
9896,941687,2024-03-05,2024-04-06,2024-04-13,28.00,Metric Ton,1.0,20' Standard,FCL,50-63,...,False,NaN,ZA,"Cape Town City Centre, Cape Town, 8000, South ...",ZA,Cape Town,US,Charleston,NaN,NaN


In [5]:
air


,create_date,valid_from,valid_to,estimated_days,origin_country,origin_name,destination_country,destination_name,shipping_type,load_type,load_amount,price_usd
0,2024-02-11,2024-02-01,2024-12-31,6.0,ZW,"Harare, Zimbabwe",MZ,"Maputo, Mozambique",RAIL_FCL,40' Standard,1,2430.00
1,2024-02-11,2024-02-01,2024-12-31,6.0,ZW,"Harare, Zimbabwe",MZ,"Maputo, Mozambique",RAIL_FCL,20' Standard,1,1850.00
2,2024-02-11,2024-01-01,2024-12-31,5.0,MZ,"Beira, Mozambique",ZW,"Harare, Zimbabwe",RAIL_FCL,40' Standard,1,1850.00
3,2024-02-11,2024-01-01,2024-12-31,5.0,MZ,"Beira, Mozambique",ZW,"Harare, Zimbabwe",RAIL_FCL,20' Standard,1,1180.00
4,2024-02-11,2024-01-01,2024-12-31,5.0,ZW,"Harare, Zimbabwe",MZ,"Beira, Mozambique",RAIL_FCL,20' Standard,1,1180.00
...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-10-09,2023-10-10,2023-10-11,NaN,US,"Los Angeles, CA, USA",US,"Via Cabrillo-Marina, San Pedro, CA 90731, USA",LTL,mt,1,293.79
996,2023-10-09,2023-11-24,2023-11-28,NaN,GB,"Victoria Embankment Gardens, Villiers St, Lond...",GB,"Cheshire, UK",LTL,mt,1,118.70
997,2023-10-09,2023-10-10,2023-10-12,NaN,US,"Los Angeles, CA, USA",US,"Via Cabrillo-Marina, San Pedro, CA 90731, USA",LTL,mt,1,135.94
998,2023-10-09,2023-11-29,2023-12-01,NaN,GB,"Victoria Embankment Gardens, Villiers St, Lond...",GB,"Cheshire, UK",LTL,mt,1,118.70


In [6]:
rates_air_renamed = air.rename(columns={
    'origin_country': 'port_from_country',
    'destination_country': 'port_to_country',
    'load_type': 'container_type'
})

# Объединение данных о запросах с тарифами для авиаперевозок.
# В реальной ситуации потребуется учитывать периоды действия тарифов, что может потребовать дополнительной обработки.
merged_leads_air = pd.merge(
    leads,
    rates_air_renamed,
    how='inner',
    on=['port_from_country', 'port_to_country', 'container_type']
)

merged_leads_air.head()


,request_id,request_date,valid_date_from,valid_date_to,weight,weight_unit,amount,container_type,shipping_type_x,hs_code,...,city_to,create_date,valid_from,valid_to,estimated_days,origin_name,destination_name,shipping_type_y,load_amount,price_usd
0,867146,2023-09-29,2023-10-01,2023-10-08,25.0,Metric Ton,1.0,40' Standard,FCL,94-96,...,NaN,2023-11-28,2023-11-28,2023-12-31,NaN,"Wuhan, Hubei, China","Madrid, Spain",RAIL_FCL,1,9900.0
1,870321,2023-10-07,2023-10-25,2023-11-01,6.0,Metric Ton,1.0,40' Standard,FCL,84-85,...,"Le Havre, France",2023-11-28,2023-11-28,2023-12-31,NaN,"Wuhan, Hubei, China","Lyon Metropolis, France",RAIL_FCL,1,9450.0
2,878371,2023-10-25,2023-11-26,2023-12-03,7.0,Metric Ton,1.0,40' Standard,FCL,482369,...,NaN,2023-11-28,2023-11-28,2023-12-31,NaN,"Wuhan, Hubei, China","Lyon Metropolis, France",RAIL_FCL,1,9450.0
3,881615,2023-11-01,2023-11-30,2023-12-07,24.0,Metric Ton,1.0,40' Standard,FCL,392062,...,NaN,2023-11-28,2023-11-28,2023-12-31,NaN,"Wuhan, Hubei, China","Madrid, Spain",RAIL_FCL,1,9900.0
4,887094,2023-11-20,2023-11-30,2023-12-07,20.0,Metric Ton,1.0,40' Standard,FCL,842951,...,"Katowice, Poland",2023-11-28,2023-11-28,2023-12-31,NaN,"Wuhan, Hubei, China","Małaszewicze, Poland",RAIL_FCL,1,7450.0


In [7]:
merged_leads_air['price_usd']

0     9900.0
1     9450.0
2     9450.0
3     9900.0
4     7450.0
5     8250.0
6     8250.0
7     9900.0
8     7450.0
9     9750.0
10    9900.0
11    9900.0
12    9900.0
13    9900.0
14    9900.0
15    9450.0
16    9750.0
17    9450.0
18    9750.0
19    7450.0
Name: price_usd, dtype: float64

In [8]:
# Подготовка и объединение данных о запросах на перевозку с тарифами для наземных перевозок.
rates_land_renamed = land.rename(columns={
    'origin_country': 'port_from_country',
    'destination_country': 'port_to_country',
    'load_type': 'container_type'
})
merged_leads_land = pd.merge(
    leads,
    rates_land_renamed,
    how='left',
    on=['port_from_country', 'port_to_country', 'container_type']
)

# Подготовка и объединение данных о запросах на перевозку с тарифами для морских перевозок.
rates_sea_renamed = sea.rename(columns={
    'origin_country': 'port_from_country',
    'destination_country': 'port_to_country',
    'load_type': 'container_type'
})
merged_leads_sea = pd.merge(
    leads,
    rates_sea_renamed,
    how='left',
    on=['port_from_country', 'port_to_country', 'container_type']
)



In [9]:
merged_leads_land['price_usd'].describe()


count    3161.000000
mean     2605.406478
std       950.939758
min        80.000000
25%      2050.000000
50%      2383.000000
75%      2950.000000
max      9394.000000
Name: price_usd, dtype: float64

In [10]:
# merged_leads_land анализ
#в основном цены только на leads_land

In [11]:
merged_leads_land['price_usd'].describe()

count    3161.000000
mean     2605.406478
std       950.939758
min        80.000000
25%      2050.000000
50%      2383.000000
75%      2950.000000
max      9394.000000
Name: price_usd, dtype: float64

In [12]:
merged_leads_land.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12554 entries, 0 to 12553
Data columns (total 35 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   request_id         12554 non-null  int64  
 1   request_date       12554 non-null  object 
 2   valid_date_from    12554 non-null  object 
 3   valid_date_to      12554 non-null  object 
 4   weight             12554 non-null  float64
 5   weight_unit        12554 non-null  object 
 6   amount             10179 non-null  float64
 7   container_type     10424 non-null  object 
 8   shipping_type_x    12554 non-null  object 
 9   hs_code            9689 non-null   object 
 10  freight_all_kinds  12554 non-null  bool   
 11  personal_cargo     12554 non-null  bool   
 12  perishable_cargo   12554 non-null  bool   
 13  liquid_cargo       12554 non-null  bool   
 14  imo_code           98 non-null     float64
 15  city_from_country  6348 non-null   object 
 16  city_from          635

In [13]:
#sepatare data for null and not null
merged_leads_land_null = merged_leads_land[merged_leads_land['price_usd'].isnull()]
merged_leads_land_not_null = merged_leads_land[merged_leads_land['price_usd'].notnull()]


In [14]:
merged_leads_land_not_null

,request_id,request_date,valid_date_from,valid_date_to,weight,weight_unit,amount,container_type,shipping_type_x,hs_code,...,valid_to,carrier,estimated_days,origin_name,origin_code5,destination_name,destination_code5,shipping_type_y,load_amount,price_usd
2,855664,2023-09-05,2023-09-05,2023-09-12,200.00,Metric Ton,1.0,20' Standard,FCL,NaN,...,2024-04-14,NaN,42.0,Shanghai,CNSHA,Caucedo,DOCAU,FCL,1.0,3467.0
3,855664,2023-09-05,2023-09-05,2023-09-12,200.00,Metric Ton,1.0,20' Standard,FCL,NaN,...,2024-04-15,MSC,31.0,Shanghai,CNSHA,Caucedo,DOCAU,FCL,1.0,3420.0
4,855664,2023-09-05,2023-09-05,2023-09-12,200.00,Metric Ton,1.0,20' Standard,FCL,NaN,...,2024-04-14,OOCL,42.0,Shanghai,CNSHA,Caucedo,DOCAU,FCL,1.0,3288.0
5,855664,2023-09-05,2023-09-05,2023-09-12,200.00,Metric Ton,1.0,20' Standard,FCL,NaN,...,2024-04-28,NaN,43.0,Shanghai,CNSHA,Caucedo,DOCAU,FCL,1.0,3087.0
6,855664,2023-09-05,2023-09-05,2023-09-12,200.00,Metric Ton,1.0,20' Standard,FCL,NaN,...,2024-04-14,NaN,42.0,Shanghai,CNSHA,Caucedo,DOCAU,FCL,1.0,3087.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12527,941621,2024-03-05,2024-03-08,2024-03-15,2.40,Metric Ton,1.0,20' Standard,FCL,851631,...,2024-04-24,ONE,51.0,Yantian,CNYTN,Southampton,GBSOU,FCL,1.0,2043.0
12546,941675,2024-03-05,2024-03-05,2024-03-12,0.76,Metric Ton,25845.0,40' Standard,FCL,50-63,...,2024-04-15,NaN,38.0,Ningbo,CNNGB,Cartagena,COCTG,FCL,1.0,3414.0
12547,941675,2024-03-05,2024-03-05,2024-03-12,0.76,Metric Ton,25845.0,40' Standard,FCL,50-63,...,2024-04-22,COSCO,39.0,Ningbo,CNNGB,Cartagena,COCTG,FCL,1.0,3400.0
12552,941690,2024-03-05,2024-04-05,2024-04-12,10.00,Metric Ton,1.0,40' Standard,FCL,84-85,...,2024-04-15,NaN,38.0,Ningbo,CNNGB,Cartagena,COCTG,FCL,1.0,3414.0


In [17]:
#Проверка форматов данных

In [18]:
merged_leads_land_not_null.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3161 entries, 2 to 12553
Data columns (total 35 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   request_id         3161 non-null   int64  
 1   request_date       3161 non-null   object 
 2   valid_date_from    3161 non-null   object 
 3   valid_date_to      3161 non-null   object 
 4   weight             3161 non-null   float64
 5   weight_unit        3161 non-null   object 
 6   amount             3161 non-null   float64
 7   container_type     3161 non-null   object 
 8   shipping_type_x    3161 non-null   object 
 9   hs_code            2459 non-null   object 
 10  freight_all_kinds  3161 non-null   bool   
 11  personal_cargo     3161 non-null   bool   
 12  perishable_cargo   3161 non-null   bool   
 13  liquid_cargo       3161 non-null   bool   
 14  imo_code           6 non-null      float64
 15  city_from_country  1375 non-null   object 
 16  city_from          1375 non-